In [1]:
import pandas as pd
df = pd.read_csv('final.csv')
df['evaluation'] = 0
df

,head_chest,chest_hip,heel-ham,elbow-shoulder,evaluation
0,0.740741,0.885167,-23,5,0
1,0.598291,0.863874,2,-6,0
2,0.550000,0.814070,18,-12,0
3,0.495868,0.829146,2,-10,0
4,0.483333,0.861538,19,-9,0
...,...,...,...,...,...
59,0.346535,0.815166,20,31,0
60,0.390000,0.801887,19,30,0
61,0.411765,0.826733,24,24,0
62,0.427184,0.806763,21,35,0


In [2]:
describe_result = df.describe()

for column in df.columns:
    lower_bound = describe_result[column]['25%']
    upper_bound = describe_result[column]['75%']
    df.loc[(df[column] >= lower_bound) & (df[column] <= upper_bound), 'evaluation'] += 1

evaluation_mapping = {
    4: 'best',
    3: 'good',
    2: 'normal',
    1: 'bad',
    0: 'worst'
}

df['evaluation'] = df['evaluation'].map(evaluation_mapping)

df

,head_chest,chest_hip,heel-ham,elbow-shoulder,evaluation
0,0.740741,0.885167,-23,5,bad
1,0.598291,0.863874,2,-6,bad
2,0.550000,0.814070,18,-12,bad
3,0.495868,0.829146,2,-10,bad
4,0.483333,0.861538,19,-9,good
...,...,...,...,...,...
59,0.346535,0.815166,20,31,normal
60,0.390000,0.801887,19,30,normal
61,0.411765,0.826733,24,24,normal
62,0.427184,0.806763,21,35,normal


In [3]:
df.describe()

,head_chest,chest_hip,heel-ham,elbow-shoulder
count,64.000000,64.000000,64.000000,64.000000
mean,0.483727,0.893624,25.171875,28.375000
std,0.099809,0.069116,14.261230,20.184467
min,0.330000,0.781991,-23.000000,-12.000000
25%,0.427251,0.829859,16.000000,13.750000
50%,0.460499,0.881670,22.500000,31.000000
75%,0.498091,0.945029,31.750000,44.000000
max,0.792683,1.028409,60.000000,77.000000


In [4]:
df['evaluation'].value_counts()

bad       22
normal    18
good      15
best       9
Name: evaluation, dtype: int64

In [5]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 데이터프레임을 로드하고 전처리
# 여기에서는 StandardScaler를 사용하여 데이터를 정규화합니다.
scaler = StandardScaler()
X = df.iloc[:, :4]
y = df['evaluation']

X = scaler.fit_transform(X)  # 데이터 정규화

# 클래스 레이블을 숫자로 매핑 (worst를 제외하고 나머지는 1씩 증가)
class_mapping = {'bad': 0, 'normal': 1, 'good': 2, 'best': 3}
y = y.map(class_mapping)

num_classes = 4  # 클래스 수 (4)를 출력 레이어의 뉴런 수로 설정
y = tf.keras.utils.to_categorical(y, num_classes)

# 클래스 레이블의 비율을 유지하며 데이터를 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 모델 아키텍처 변경
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(4,)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')  # 출력 뉴런 수를 클래스 수와 일치시킴
])

# 학습률 조정
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

model.fit(X_train, y_train, epochs=100, batch_size=6, validation_data=(X_test, y_test))

accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy[1]}')

Epoch 1/100
9/9 [==============================] - 1s 44ms/step - loss: 1.3243 - accuracy: 0.3333 - val_loss: 1.3460 - val_accuracy: 0.4615
Epoch 2/100
9/9 [==============================] - 0s 4ms/step - loss: 1.2558 - accuracy: 0.4118 - val_loss: 1.3206 - val_accuracy: 0.4615
Epoch 3/100
9/9 [==============================] - 0s 5ms/step - loss: 1.2125 - accuracy: 0.4510 - val_loss: 1.3123 - val_accuracy: 0.4615
Epoch 4/100
9/9 [==============================] - 0s 5ms/step - loss: 1.1752 - accuracy: 0.4118 - val_loss: 1.3035 - val_accuracy: 0.3846
Epoch 5/100
9/9 [==============================] - 0s 6ms/step - loss: 1.1405 - accuracy: 0.4706 - val_loss: 1.3043 - val_accuracy: 0.3846
Epoch 6/100
9/9 [==============================] - 0s 6ms/step - loss: 1.1150 - accuracy: 0.4706 - val_loss: 1.3052 - val_accuracy: 0.4615
Epoch 7/100
9/9 [==============================] - 0s 4ms/step - loss: 1.0965 - accuracy: 0.5686 - val_loss: 1.3122 - val_accuracy: 0.3846
Epoch 8/100
9/9 [=========

9/9 [==============================] - 0s 5ms/step - loss: 0.6413 - accuracy: 0.8039 - val_loss: 0.8660 - val_accuracy: 0.6923
Epoch 60/100
9/9 [==============================] - 0s 5ms/step - loss: 0.6339 - accuracy: 0.8235 - val_loss: 0.8454 - val_accuracy: 0.6923
Epoch 61/100
9/9 [==============================] - 0s 7ms/step - loss: 0.6284 - accuracy: 0.8235 - val_loss: 0.8371 - val_accuracy: 0.6923
Epoch 62/100
9/9 [==============================] - 0s 5ms/step - loss: 0.6185 - accuracy: 0.8235 - val_loss: 0.8413 - val_accuracy: 0.6923
Epoch 63/100
9/9 [==============================] - 0s 5ms/step - loss: 0.6133 - accuracy: 0.8039 - val_loss: 0.8463 - val_accuracy: 0.6923
Epoch 64/100
9/9 [==============================] - 0s 4ms/step - loss: 0.6033 - accuracy: 0.8039 - val_loss: 0.8457 - val_accuracy: 0.6923
Epoch 65/100
9/9 [==============================] - 0s 4ms/step - loss: 0.5981 - accuracy: 0.8039 - val_loss: 0.8271 - val_accuracy: 0.6923
Epoch 66/100
9/9 [===============

In [6]:
model.save("my_model.h5")

C:\anaconda3\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [7]:
model.save('my_model.keras')